<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [ ]:
# Install spark

In [ ]:
!pip install pyspark
!pip install findspark

In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [ ]:
# Start session

In [3]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Saving and Loading a SparkML Model").getOrCreate()

25/03/27 23:09:12 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/27 23:09:14 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [2]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

--2025-03-27 23:08:04--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv’

searchterms.csv     100%[===================>] 227.99K  --.-KB/s    in 0.004s  

2025-03-27 23:08:04 (55.1 MB/s) - ‘searchterms.csv’ saved [233457/233457]



In [ ]:
# Load the csv into a spark dataframe

In [43]:
df = spark.read.csv("searchterms.csv", inferSchema=True, header=True)

In [ ]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [44]:
# Get number of rows and columns
num_rows = df.count()
num_cols = len(df.columns)

# Print shape (similar to pandas)
print((num_rows, num_cols))

(10000, 4)


In [ ]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [45]:
df.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [ ]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [46]:
df.printSchema()

root
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- searchterm: string (nullable = true)



In [ ]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [48]:
df.createTempView("searchterm")
query =  "SELECT count(*) FROM searchterm WHERE searchterm like '%gaming laptop%'"
results = spark.sql(query)
results.show()

+--------+
|count(1)|
+--------+
|     499|
+--------+



In [ ]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [ ]:
query =  """
SELECT searchterm, count(*) AS frequency FROM searchterm 
GROUP BY searchterm ORDER BY frequency DESC"""
results = spark.sql(query)
results.show(5)

In [ ]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
!tar -xvzf model.tar.gz
!ls

In [ ]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [36]:
from pyspark.ml.regression import LinearRegressionModel
# Load persisted model
loaded_model = LinearRegressionModel.load("sales_prediction.model")

In [ ]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [42]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
assembler = VectorAssembler(inputCols=["year"], outputCol="features")
data = [[2023,0]]
columns = ["year", "sales"]
df = spark.createDataFrame(data, columns)
df = assembler.transform(df).select("features","sales")
pred = loaded_model.transform(df)
pred.select("prediction").show()

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+



25/03/27 23:46:45 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
25/03/27 23:46:45 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [ ]:
spark.stop